In [1]:
import os
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "0"

In [ ]:
from unsloth import FastLanguageModel
import os
import torch
max_seq_length = 2048 # Qualsiasi valore, dato che viene effettuato RoPE Scaling in automatico.
dtype = None # None per auto detection.
load_in_4bit = True # Carica il modello quantizzato in 4bit.
lora_rank = 64 # Larger rank = smarter, but slower

hf_token = os.environ.get("HF_TOKEN") or os.environ.get("HUGGINGFACE_HUB_TOKEN")

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Alelcv27/Llama3.1-8B-Code",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = hf_token,
    max_lora_rank=lora_rank
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/user/.pyenv/versions/uns2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.3: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    NVIDIA GeForce RTX 3090. Num GPUs = 1. Max memory: 23.676 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth


Loading checkpoint shards: 100%|██████████| 4/4 [00:09<00:00,  2.44s/it]


In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = lora_rank,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = True,
    random_state = 3407,
    use_rslora = True,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.11.3 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [4]:
from datasets import load_dataset, Dataset

# Load your dataset - update the dataset name/path as needed
# The dataset should have "problem" and "generated_solution" columns
# Example: dataset = load_dataset("your_dataset_name", split="train")
# Or: dataset = load_dataset("path/to/local/dataset")

# Load streaming dataset and convert to regular dataset
print("Loading streaming dataset...")
dataset_streaming = load_dataset("nvidia/OpenMathReasoning", split="cot", streaming=True).take(2000)

# Convert streaming dataset to list, then to regular Dataset
print("Converting to regular dataset...")
dataset_list = list(dataset_streaming)
dataset = Dataset.from_list(dataset_list)

# If your dataset has different column names, map them here
# For example, if columns are "question" and "answer":
# dataset = dataset.rename_columns({"question": "problem", "answer": "generated_solution"})
original_columns = dataset.column_names
print(f"Dataset columns: {original_columns}")
print(f"Dataset size: {len(dataset)}")
print(f"First example keys: {list(dataset[0].keys())}")

Loading streaming dataset...
Converting to regular dataset...
Dataset columns: ['expected_answer', 'problem_type', 'problem_source', 'generation_model', 'pass_rate_72b_tir', 'problem', 'generated_solution', 'inference_mode', 'used_in_kaggle']
Dataset size: 2000
First example keys: ['expected_answer', 'problem_type', 'problem_source', 'generation_model', 'pass_rate_72b_tir', 'problem', 'generated_solution', 'inference_mode', 'used_in_kaggle']


In [5]:
# Alpaca prompt format with system prompt, problem, and solution
alpaca_prompt = """Solve problems by breaking them down into logical steps, explaining your methodology and reasoning for each part of the process to provide a clear, comprehensive solution.

### Problem:
{problem}

### Solution:
{solution}"""


EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    problems = examples["problem"]
    solutions = examples["generated_solution"]
    texts = []
    for problem, solution in zip(problems, solutions):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(problem=problem, solution=solution) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

In [6]:
original_columns = dataset.column_names
dataset = dataset.map(formatting_prompts_func, batched=True, remove_columns=original_columns)

print(dataset[0].get('text', 'No text field found')[:500] + "..." if len(dataset[0].get('text', '')) > 500 else dataset[0].get('text', 'No text field found'))

Map: 100%|██████████| 2000/2000 [00:00<00:00, 11667.86 examples/s]

Solve problems by breaking them down into logical steps, explaining your methodology and reasoning for each part of the process to provide a clear, comprehensive solution.

### Problem:
Given a group of \( N \) balls consisting of \( C \) colors, where the number of balls in each color is represented as \( n_1, n_2, \ldots, n_C \) (with \( n_1 + n_2 + \ldots + n_C = N \)), what is the probability that when \( A \) balls are randomly picked (where \( A \leq N \)), the picked balls consist of \( a...


In [7]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text", # Specifica il nome del campo nel dataset in cui è archiviato il testo di input
    max_seq_length = max_seq_length,
    dataset_num_proc = 2, # Numero di processi da utilizzare per la pre-elaborazione del dataset
    packing = True,
    args = TrainingArguments(
        per_device_train_batch_size = 4, # Numero di training sample per batch per device (GPU/CPU)
        gradient_accumulation_steps = 4, # Il numero di step per accumulare i gradienti prima di eseguire un backward pass.
        max_grad_norm=1.0,
        warmup_ratio=0.05, # Numero di step iniziali durante i quali il learning rate aumenta linearmente da 0 al valore impostato.
        num_train_epochs = 1,
        learning_rate = 1e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 10, # Con quale frequenza (in termini di step) devono essere stampati i log di training.
        optim = "adamw_8bit", # Optimizer utilizzato
        weight_decay = 0.01,
        lr_scheduler_type = "cosine", # Tipo di learning rate scheduler da utilizzare
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=28): 100%|██████████| 2000/2000 [00:10<00:00, 185.29 examples/s]


In [8]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 3090. Max memory = 23.676 GB.
7.137 GB of memory reserved.


In [9]:
trainer_stats = trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,000 | Num Epochs = 1 | Total steps = 125
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 167,772,160 of 8,198,033,408 (2.05% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,0.990000
20,0.763400
30,0.736500
40,0.699900
50,0.686800
60,0.729100
70,0.682200
80,0.688900
90,0.679800
100,0.672300


In [10]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

1623.2553 seconds used for training.
27.05 minutes used for training.
Peak reserved memory = 10.088 GB.
Peak reserved memory for training = 2.951 GB.
Peak reserved memory % of max memory = 42.609 %.
Peak reserved memory for training % of max memory = 12.464 %.


In [11]:
FastLanguageModel.for_inference(model)

inputs = tokenizer(
[
    alpaca_prompt.format(
        problem = "For the natural number A, the quotient of A divided by 9 is 6 and the remainder is 5. What is the value of A?",
        solution = "" # Leave this empty for the model to fill
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer)

<|begin_of_text|>Solve problems by breaking them down into logical steps, explaining your methodology and reasoning for each part of the process to provide a clear, comprehensive solution.

### Problem:
For the natural number A, the quotient of A divided by 9 is 6 and the remainder is 5. What is the value of A?

### Solution:
<think>
Okay, so I need to find the value of A where when you divide it by 9, the quotient is 6 and the remainder is 5. Hmm, let me think. 

First, I remember that when you divide a number by another, you can write it as A = 9*6 + 5. Because the quotient is 6, so that's 9 times 6, and then you add the remainder, which is 5. Let me check that. If I do 9*6, that's 54, and then add 5, that's 59. So A would be 59. But wait, is that right? Let me make sure.

Alternatively, maybe I should use the division algorithm. The division algorithm says that for any integer A and positive integer d, there exist unique integers q and r such that A = d*q + r and 0 ≤ r < d. Here, d 

In [12]:
output_repo = "Alelcv27/Llama3.1-8B-Code-Math"

model.push_to_hub_merged(output_repo, tokenizer, save_method = "merged_16bit", token = token)

Processing Files (1 / 1): 100%|██████████| 17.2MB / 17.2MB, 5.38MB/s  
New Data Upload: |          |  0.00B /  0.00B,  0.00B/s  


Found HuggingFace hub cache directory: /home/user/.cache/huggingface/hub


Fetching 1 files: 100%|██████████| 1/1 [00:00<00:00,  4.22it/s]
No files have been modified since last commit. Skipping to prevent empty commit.
[huggingface_hub.hf_api|WARNING]No files have been modified since last commit. Skipping to prevent empty commit.


Checking cache directory for required files...


Unsloth: Copying 4 files from cache to `Alelcv27/Llama3.1-8B-Code-Math`: 100%|██████████| 4/4 [00:08<00:00,  2.23s/it]


Successfully copied all 4 files from cache to `Alelcv27/Llama3.1-8B-Code-Math`
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Preparing safetensor model files: 100%|██████████| 4/4 [00:00<00:00, 93727.46it/s]
Processing Files (1 / 1): 100%|██████████| 4.98GB / 4.98GB,  848kB/s  t/s]
New Data Upload: 100%|██████████| 3.93GB / 3.93GB,  848kB/s  
Processing Files (1 / 1): 100%|██████████| 5.00GB / 5.00GB, 1.65MB/s  , 323.41s/it]
New Data Upload: 100%|██████████| 5.00GB / 5.00GB, 1.65MB/s  
Processing Files (1 / 1): 100%|██████████| 4.92GB / 4.92GB,  0.00B/s  , 367.23s/it]
New Data Upload: 100%|██████████| 4.92GB / 4.92GB,  0.00B/s  
Processing Files (1 / 1): 100%|██████████| 1.17GB / 1.17GB, 4.25MB/s  , 379.67s/it]
New Data Upload: 100%|██████████|  118MB /  118MB, 4.25MB/s  
Unsloth: Merging weights into 16bit: 100%|██████████| 4/4 [19:07<00:00, 286.97s/it]


Unsloth: Merge process complete. Saved to `/home/user/Progetti/Merging/llama3 notebooks/Alelcv27/Llama3.1-8B-Code-Math`
